In [1]:
import os

In [2]:
%pwd

'/Users/macbookpro/Desktop/MLProjects/Text-Summarizer-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/macbookpro/Desktop/MLProjects/Text-Summarizer-Project'

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [8]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

/Applications/anaconda3/envs/textSumm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-09-02 23:10:53,465: INFO: config: PyTorch version 2.2.2 available.]


In [13]:
class DataTransformation:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        create_directories([self.config.data_transformation.root_dir])

        self.tokenizer = AutoTokenizer.from_pretrained(self.config.data_transformation.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_transformation.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.data_transformation.root_dir,"samsum_dataset"))

In [14]:
try:
    data_transformation = DataTransformation()
    data_transformation.convert()
except Exception as e:
    raise e

[2025-09-02 23:12:21,768: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-02 23:12:21,779: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-02 23:12:21,781: INFO: common: created directory at: artifacts]
[2025-09-02 23:12:21,783: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/Applications/anaconda3/envs/textSumm/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 819/819 [00:00<00:00, 8473.78 examples/s]
